In [1]:
#tensorいじりのテストを行う
import torch
import torch.nn
x = torch.randn(10,10,10)
print(torch.cuda.is_available())
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
from audio_to_multiple_pose_gan.config import get_config
import warnings
warnings.simplefilter('ignore')

True


In [2]:
import pandas as pd
from common.consts import AUDIO_SHAPE
batch_size = 32
train_csv: str = "~/Desktop/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#train_csv: str = "~/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#df = pd.read_csv(self.args.train_csv)
#cfg = get_config(self.args.config)
df = pd.read_csv(train_csv)
#cfg = get_config(config)
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [3]:
from common.consts import AUDIO_SHAPE
configs = {
    "audio_to_pose": {"num_keypoints": 98, "processor": "audio_to_pose", "flatten": False, "input_shape": [None, AUDIO_SHAPE]},
    "audio_to_pose_inference": {"num_keypoints": 98, "processor": "audio_to_pose_inference", "flatten": False, "input_shape": [None, AUDIO_SHAPE]}
}
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [4]:
row = df.sample(n=1).iloc[0]
X, Y = generate_batch(df, process_row, batch_size)

In [5]:
Y.shape

(32, 64, 98)

In [6]:
X.shape

(32, 67267)

In [7]:
df.sample(n=1).iloc[0]

dataset                                                    train
start                                            00:19:47.020354
end                                              00:19:51.224558
interval_id                                                  159
pose_fn        Gestures/shelly/train/npz/159-00:19:47.020354-...
audio_fn       Gestures/shelly/train/audio/159-00:19:33.34000...
video_fn       17._The_badness_of_death_Part_II_-_The_depriva...
speaker                                                   shelly
Name: 63912, dtype: object

In [8]:
from audio_to_multiple_pose_gan.static_model_factory import *
a2p = Audio2Pose(1, 98).to(device)
a2p(torch.tensor(X).float().to(device)).shape

torch.Size([32, 64, 98])

In [9]:
Y.shape

(32, 64, 98)

In [10]:
d_patchgan = D_patchgan(in_channels=64)

In [11]:
Y.shape

(32, 64, 98)

In [12]:
Y.shape

(32, 64, 98)

In [13]:
from audio_to_multiple_pose_gan.torch_layers import *
to_motion_delta(Y).shape

(32, 63, 98)

In [14]:
Y = torch.tensor(Y).float().to(device)

In [15]:
a2p = Audio2PoseGANS(1, 98).to(device)
pred_Y = a2p(torch.tensor(X).float().to(device))

In [16]:
keypoints_regloss(Y, pred_Y, regloss_type="l1")

tensor(0.7741, device='cuda:0', grad_fn=<MeanBackward0>)

In [17]:
from audio_to_multiple_pose_gan.config import get_config
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
#get_modelの実装は任せる
from audio_to_multiple_pose_gan.static_model_factory import Audio2PoseGANS, D_patchgan
from audio_to_multiple_pose_gan.static_model_factory import Audio2PoseGANSTransformer
from audio_to_multiple_pose_gan.torch_layers import to_motion_delta, keypoints_to_train, keypoints_regloss
from common.audio_lib import save_audio_sample
from common.audio_repr import raw_repr
from common.consts import RIGHT_BODY_KEYPOINTS, LEFT_BODY_KEYPOINTS, LEFT_HAND_KEYPOINTS, \
    RIGHT_HAND_KEYPOINTS, POSE_SAMPLE_SHAPE, G_SCOPE, D_SCOPE, SR, FRAMES_PER_SAMPLE, AUDIO_SHAPE
from common.evaluation import compute_pck
from common.pose_logic_lib import translate_keypoints, get_sample_output_by_config
from common.pose_plot_lib import save_side_by_side_video, save_video_from_audio_video
from torch import optim
import torch
import pandas as pd
import argparse
from torch import nn

def get_training_keypoints():
    training_keypoints = []
    training_keypoints.extend(RIGHT_BODY_KEYPOINTS)
    training_keypoints.extend(LEFT_BODY_KEYPOINTS)
    for i in range(5):
        training_keypoints.extend(RIGHT_HAND_KEYPOINTS(i))
        training_keypoints.extend(LEFT_HAND_KEYPOINTS(i))
    training_keypoints = sorted(list(set(training_keypoints)))
    return training_keypoints

In [18]:
poses = decode_pose(Y.cpu().numpy()[0], shift = [0,0], speaker="shelly")

In [19]:
G_model = Audio2PoseGANSTransformer(1, 98).to(device)

In [20]:
state_dict = torch.load("params/G_model_last.pth")

In [21]:
from collections import OrderedDict
param_dict = OrderedDict()
for key in state_dict:
    param_dict[key.replace("_orig_mod.","")] = state_dict[key]

In [22]:
G_model(torch.tensor(X).float().to(device))

tensor([[[-0.1909, -0.0718,  0.3732,  ...,  0.0919,  0.2334, -0.1474],
         [-0.1898, -0.1460, -0.0190,  ..., -0.1023,  0.4728,  0.4190],
         [ 0.3253, -0.3330,  0.6255,  ...,  0.1428,  0.4928,  0.2183],
         ...,
         [ 0.6822, -0.5046,  0.3550,  ...,  0.2125,  0.1474,  0.1614],
         [ 0.4125, -0.4230,  0.3236,  ..., -0.7149,  0.1827,  0.4612],
         [ 0.6814, -0.0091, -0.2126,  ...,  0.0538, -0.2648,  0.2698]],

        [[ 0.2577, -0.2946, -0.0313,  ...,  0.3618,  0.2569,  0.0670],
         [ 0.2711, -0.1868, -0.3613,  ..., -0.1652, -0.3742, -0.1795],
         [ 0.3937, -0.0010, -0.1174,  ..., -0.8267, -0.3643,  0.1580],
         ...,
         [ 0.4137, -0.5900, -0.6363,  ..., -0.0612,  0.4452,  1.0174],
         [-0.1192,  0.1979, -0.6025,  ..., -0.1687, -0.2689,  0.1908],
         [-0.1339, -0.3159, -0.1721,  ..., -0.2002, -0.0370, -0.0074]],

        [[ 0.3284,  0.0037,  0.5881,  ..., -0.1123,  0.1763, -0.1282],
         [ 0.2052,  0.3829,  0.0132,  ...,  0

In [23]:
cfg: dict = {"processor": "audio_to_pose_new_keypoints", "input_shape": [None, AUDIO_SHAPE], "new_keypoints":True}
process_row, decode_pose = get_processor(cfg)
row = df.sample(n=1).iloc[0]

In [24]:
X, Y = process_row(row) #増えた次元の7, 8, 9がnose, eyesとなる。

In [26]:
import numpy as np
x = np.random.randn(64,2,49)

In [32]:
X, Y = generate_batch(df, process_row, batch_size)


In [34]:
X.shape

(32, 67267)